In [46]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import tensorflow as tf
import time

# to make this notebook's output stable across runs
def reset_graph(seed=42): 
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

reset_graph()

In [47]:
# load data: digits 5 to 9, but still label with 0 to 4, 
# because TensorFlow expects label's integers from 0 to n_classes-1.
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

X_train2_full = mnist.train.images[mnist.train.labels >= 5]
y_train2_full = mnist.train.labels[mnist.train.labels >= 5] - 5
X_valid2_full = mnist.validation.images[mnist.validation.labels >= 5]
y_valid2_full = mnist.validation.labels[mnist.validation.labels >= 5] - 5
X_test2 = mnist.test.images[mnist.test.labels >= 5]
y_test2 = mnist.test.labels[mnist.test.labels >= 5] - 5

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [48]:
# we want to keep only 100 instances per class in the training set 
# and let's keep only 30 instances per class in the validation set
# tesing set is already loaded above
def sample_n_instances_per_class(X, y, n=100):
    Xs, ys = [], []
    for label in np.unique(y):
        idx = (y == label)
        Xc = X[idx][:n]
        yc = y[idx][:n]
        Xs.append(Xc)
        ys.append(yc)
    return np.concatenate(Xs), np.concatenate(ys)

X_train2, y_train2 = sample_n_instances_per_class(X_train2_full, y_train2_full, n=100)
X_valid2, y_valid2 = sample_n_instances_per_class(X_valid2_full, y_valid2_full, n=30)

In [ ]:
learning_rate = 0.001
batch_size = 16
train_size = X_train2.shape[0]

#saver = tf.train.Saver()

with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('/Team48_HW2.ckpt.meta', clear_devices=True)
    new_saver.restore(sess, '/Team48_HW2.ckpt')
    xs = tf.get_default_graph().get_tensor_by_name("xs:0")
    ys = tf.get_default_graph().get_tensor_by_name("ys:0")
    y = tf.get_default_graph().get_tensor_by_name("y:0")
    #logits = y.op.inputs[0]
    accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
    loss = tf.get_default_graph().get_tensor_by_name("loss:0")
    optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars, name="training_op")
    
    tf.global_variables_initializer().run() 
    
    best_loss = 99999
    stand_epoch = 0
    
    t0 = time.time()
    
    for i in range(1000):
        #sess.run(training_op, feed_dict={xs: X_train2, ys: y_train2})
        stand_epoch = stand_epoch + 1
        for j in range(int(train_size/batch_size)):
            # get batch data each time
            batch_x = X_train2[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(training_op, feed_dict={xs: batch_x, ys: batch_y})
        
        val_loss = sess.run(loss, feed_dict={xs: X_valid2, ys: y_valid2})
        if best_loss > val_loss:
            stand_epoch = 0
            best_loss = val_loss
            
        
        accu = sess.run(accuracy, feed_dict={xs: X_test2, ys: y_test2})
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu)
        
        if stand_epoch >= 20: # Early stop
            print("Early stop in epoch", i)
            break
    t1 = time.time()
    save_path = new_saver.save(sess, "/Team48_HW3_1.ckpt") # save variables to checkpoint

sess.close()
print("Training time:", round(t1-t0,2), "sec")
print("Training Finish!!!")

INFO:tensorflow:Restoring parameters from /Team48_HW2.ckpt
0 Validation loss: 232.55  ; Best loss: 232.55  ; Accuracy: 0.333882
1 Validation loss: 225.232  ; Best loss: 225.232  ; Accuracy: 0.422547
2 Validation loss: 218.671  ; Best loss: 218.671  ; Accuracy: 0.50504
3 Validation loss: 212.781  ; Best loss: 212.781  ; Accuracy: 0.570253
4 Validation loss: 207.636  ; Best loss: 207.636  ; Accuracy: 0.606254
5 Validation loss: 203.254  ; Best loss: 203.254  ; Accuracy: 0.63238
6 Validation loss: 199.473  ; Best loss: 199.473  ; Accuracy: 0.653569
7 Validation loss: 196.166  ; Best loss: 196.166  ; Accuracy: 0.668175
8 Validation loss: 193.308  ; Best loss: 193.308  ; Accuracy: 0.679696
9 Validation loss: 190.892  ; Best loss: 190.892  ; Accuracy: 0.691216
10 Validation loss: 188.864  ; Best loss: 188.864  ; Accuracy: 0.696976
11 Validation loss: 187.146  ; Best loss: 187.146  ; Accuracy: 0.704588
12 Validation loss: 185.672  ; Best loss: 185.672  ; Accuracy: 0.710965
13 Validation loss:

115 Validation loss: 165.826  ; Best loss: 165.826  ; Accuracy: 0.811561
116 Validation loss: 165.792  ; Best loss: 165.792  ; Accuracy: 0.812179
117 Validation loss: 165.758  ; Best loss: 165.758  ; Accuracy: 0.812796
118 Validation loss: 165.725  ; Best loss: 165.725  ; Accuracy: 0.813001
119 Validation loss: 165.691  ; Best loss: 165.691  ; Accuracy: 0.813619
120 Validation loss: 165.658  ; Best loss: 165.658  ; Accuracy: 0.81403
121 Validation loss: 165.624  ; Best loss: 165.624  ; Accuracy: 0.813824
122 Validation loss: 165.59  ; Best loss: 165.59  ; Accuracy: 0.814853
123 Validation loss: 165.555  ; Best loss: 165.555  ; Accuracy: 0.814853
124 Validation loss: 165.52  ; Best loss: 165.52  ; Accuracy: 0.815264
125 Validation loss: 165.485  ; Best loss: 165.485  ; Accuracy: 0.815676
126 Validation loss: 165.449  ; Best loss: 165.449  ; Accuracy: 0.816499
127 Validation loss: 165.413  ; Best loss: 165.413  ; Accuracy: 0.815676
128 Validation loss: 165.377  ; Best loss: 165.377  ; Ac

231 Validation loss: 162.289  ; Best loss: 162.289  ; Accuracy: 0.826373
232 Validation loss: 162.273  ; Best loss: 162.273  ; Accuracy: 0.826373
233 Validation loss: 162.258  ; Best loss: 162.258  ; Accuracy: 0.826373
234 Validation loss: 162.243  ; Best loss: 162.243  ; Accuracy: 0.826579
235 Validation loss: 162.228  ; Best loss: 162.228  ; Accuracy: 0.826579
236 Validation loss: 162.214  ; Best loss: 162.214  ; Accuracy: 0.826579
237 Validation loss: 162.201  ; Best loss: 162.201  ; Accuracy: 0.826579
238 Validation loss: 162.187  ; Best loss: 162.187  ; Accuracy: 0.826579
239 Validation loss: 162.175  ; Best loss: 162.175  ; Accuracy: 0.827402
240 Validation loss: 162.162  ; Best loss: 162.162  ; Accuracy: 0.827607
241 Validation loss: 162.15  ; Best loss: 162.15  ; Accuracy: 0.828225
242 Validation loss: 162.139  ; Best loss: 162.139  ; Accuracy: 0.828225
243 Validation loss: 162.128  ; Best loss: 162.128  ; Accuracy: 0.828225
244 Validation loss: 162.117  ; Best loss: 162.117  ;

In [43]:
tf.reset_default_graph() 
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('/Team48_HW2.ckpt.meta', clear_devices=True)
    new_saver.restore(sess, '/Team48_HW2.ckpt')
    xs = tf.get_default_graph().get_tensor_by_name("xs:0")
    ys = tf.get_default_graph().get_tensor_by_name("ys:0")
    y = tf.get_default_graph().get_tensor_by_name("y:0")
    accuracy = tf.get_default_graph().get_tensor_by_name("accuracy:0")
    
    layer5 = tf.get_default_graph().get_tensor_by_name("l5:0")
    output_layer_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope="logits")
    loss = tf.get_default_graph().get_tensor_by_name("loss:0")
    optimizer = tf.train.AdamOptimizer(learning_rate, name="optimizer")
    training_op = optimizer.minimize(loss, var_list=output_layer_vars, name="training_op")
    
    tf.global_variables_initializer().run()
    X_train_val = np.concatenate((X_train2, X_valid2))
    layer5_cache = sess.run(layer5, feed_dict={xs: X_train_val, ys: y_train2})
    
    best_loss = 99999
    stand_epoch = 0
    
    t3 = time.time()
    for i in range(1000):
        
        stand_epoch = stand_epoch + 1
        for j in range(int(train_size/batch_size)):
            # get batch data each time
            batch_x = layer5_cache[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(training_op, feed_dict={layer5: batch_x, ys: batch_y})
        
        val_loss = sess.run(loss, feed_dict={layer5: layer5_cache[500:,:], ys: y_valid2})
        if best_loss > val_loss:
            stand_epoch = 0
            best_loss = val_loss
            
        accu = sess.run(accuracy, feed_dict={xs: X_test2, ys: y_test2})
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu)
        
        if stand_epoch >= 20: # Early stop
            print("Early stop in epoch", i)
            break
    
    t4 = time.time()
    save_path = new_saver.save(sess, "/Team48_HW3_2.ckpt") # save variables to checkpoint

print(round(t4-t3,2))
print("Training Finish!!!")
sess.close()

INFO:tensorflow:Restoring parameters from /Team48_HW2.ckpt
0 Validation loss: 232.55  ; Best loss: 232.55  ; Accuracy: 0.333882
1 Validation loss: 225.232  ; Best loss: 225.232  ; Accuracy: 0.422547
2 Validation loss: 218.671  ; Best loss: 218.671  ; Accuracy: 0.50504
3 Validation loss: 212.781  ; Best loss: 212.781  ; Accuracy: 0.570253
4 Validation loss: 207.636  ; Best loss: 207.636  ; Accuracy: 0.606254
5 Validation loss: 203.254  ; Best loss: 203.254  ; Accuracy: 0.63238
6 Validation loss: 199.473  ; Best loss: 199.473  ; Accuracy: 0.653569
7 Validation loss: 196.166  ; Best loss: 196.166  ; Accuracy: 0.668175
8 Validation loss: 193.308  ; Best loss: 193.308  ; Accuracy: 0.679696
9 Validation loss: 190.892  ; Best loss: 190.892  ; Accuracy: 0.691216
10 Validation loss: 188.864  ; Best loss: 188.864  ; Accuracy: 0.696976
11 Validation loss: 187.146  ; Best loss: 187.146  ; Accuracy: 0.704588
12 Validation loss: 185.672  ; Best loss: 185.672  ; Accuracy: 0.710965
13 Validation loss:

114 Validation loss: 165.86  ; Best loss: 165.86  ; Accuracy: 0.811356
115 Validation loss: 165.826  ; Best loss: 165.826  ; Accuracy: 0.811561
116 Validation loss: 165.792  ; Best loss: 165.792  ; Accuracy: 0.812178
117 Validation loss: 165.758  ; Best loss: 165.758  ; Accuracy: 0.812796
118 Validation loss: 165.725  ; Best loss: 165.725  ; Accuracy: 0.813001
119 Validation loss: 165.691  ; Best loss: 165.691  ; Accuracy: 0.813619
120 Validation loss: 165.658  ; Best loss: 165.658  ; Accuracy: 0.81403
121 Validation loss: 165.624  ; Best loss: 165.624  ; Accuracy: 0.813824
122 Validation loss: 165.59  ; Best loss: 165.59  ; Accuracy: 0.814853
123 Validation loss: 165.555  ; Best loss: 165.555  ; Accuracy: 0.814853
124 Validation loss: 165.52  ; Best loss: 165.52  ; Accuracy: 0.815264
125 Validation loss: 165.485  ; Best loss: 165.485  ; Accuracy: 0.815676
126 Validation loss: 165.449  ; Best loss: 165.449  ; Accuracy: 0.816499
127 Validation loss: 165.413  ; Best loss: 165.413  ; Accu

228 Validation loss: 162.338  ; Best loss: 162.338  ; Accuracy: 0.826167
229 Validation loss: 162.321  ; Best loss: 162.321  ; Accuracy: 0.825756
230 Validation loss: 162.305  ; Best loss: 162.305  ; Accuracy: 0.826373
231 Validation loss: 162.289  ; Best loss: 162.289  ; Accuracy: 0.826373
232 Validation loss: 162.273  ; Best loss: 162.273  ; Accuracy: 0.826373
233 Validation loss: 162.258  ; Best loss: 162.258  ; Accuracy: 0.826373
234 Validation loss: 162.243  ; Best loss: 162.243  ; Accuracy: 0.826579
235 Validation loss: 162.228  ; Best loss: 162.228  ; Accuracy: 0.826579
236 Validation loss: 162.214  ; Best loss: 162.214  ; Accuracy: 0.826579
237 Validation loss: 162.201  ; Best loss: 162.201  ; Accuracy: 0.826579
238 Validation loss: 162.187  ; Best loss: 162.187  ; Accuracy: 0.826579
239 Validation loss: 162.175  ; Best loss: 162.175  ; Accuracy: 0.827402
240 Validation loss: 162.162  ; Best loss: 162.162  ; Accuracy: 0.827607
241 Validation loss: 162.15  ; Best loss: 162.15  ;

In [45]:
tf.reset_default_graph() 
with tf.Session() as sess:
    new_saver = tf.train.import_meta_graph('/Team48_HW2.ckpt.meta', clear_devices=True)
    new_saver.restore(sess, '/Team48_HW2.ckpt')
    xs = tf.get_default_graph().get_tensor_by_name("xs:0")
    ys = tf.get_default_graph().get_tensor_by_name("ys:0")
    
    layer4 = tf.get_default_graph().get_tensor_by_name("l4:0")
    Weights = tf.get_variable("third_logits", shape=[128, 5], initializer=tf.contrib.layers.variance_scaling_initializer())
    third_logits = tf.matmul(layer4, Weights)
    y = tf.nn.softmax(third_logits, name="third_y")
    cross_entropy = tf.reduce_sum(tf.nn.sparse_softmax_cross_entropy_with_logits(labels = ys, logits = y), name="third_loss") # Cost
    optm = tf.train.AdamOptimizer(learning_rate, name="optm")
    train_step = optm.minimize(cross_entropy, var_list=Weights, name="training_step") # Optimizer
    accuracy_rate = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), ys), tf.float32), name = "third_accuracy") # Calculate accuracy rate
    
    tf.global_variables_initializer().run() 
    
    best_loss = 99999
    stand_epoch = 0
    
    t5 = time.time()
    
    for i in range(1000):
        stand_epoch = stand_epoch + 1
        for j in range(int(train_size/batch_size)):
            # get batch data each time
            batch_x = X_train2[j*batch_size:(j+1)*batch_size,:]
            batch_y = y_train2[j*batch_size:(j+1)*batch_size]
            # training
            sess.run(train_step, feed_dict={xs: batch_x, ys: batch_y})
        
        val_loss = sess.run(cross_entropy, feed_dict={xs: X_valid2, ys: y_valid2})
        if best_loss > val_loss:
            stand_epoch = 0
            best_loss = val_loss
            
        
        accu = sess.run(accuracy_rate, feed_dict={xs: X_test2, ys: y_test2})
        print(i,"Validation loss:", val_loss, " ; Best loss:", best_loss, " ; Accuracy:", accu)
        
        if stand_epoch >= 20: # Early stop
            print("Early stop in epoch", i)
            break
    t6 = time.time()
    save_path = new_saver.save(sess, "/Team48_HW3_1.ckpt") # save variables to checkpoint

sess.close()
print("Training time:", round(t6-t5,2), "sec")
print("Training Finish!!!")

INFO:tensorflow:Restoring parameters from /Team48_HW2.ckpt
0 Validation loss: 237.154  ; Best loss: 237.154  ; Accuracy: 0.284509
1 Validation loss: 230.616  ; Best loss: 230.616  ; Accuracy: 0.340671
2 Validation loss: 223.82  ; Best loss: 223.82  ; Accuracy: 0.400741
3 Validation loss: 217.351  ; Best loss: 217.351  ; Accuracy: 0.447233
4 Validation loss: 211.945  ; Best loss: 211.945  ; Accuracy: 0.480148
5 Validation loss: 207.353  ; Best loss: 207.353  ; Accuracy: 0.520058
6 Validation loss: 203.177  ; Best loss: 203.177  ; Accuracy: 0.549887
7 Validation loss: 199.326  ; Best loss: 199.326  ; Accuracy: 0.588151
8 Validation loss: 195.875  ; Best loss: 195.875  ; Accuracy: 0.623534
9 Validation loss: 192.894  ; Best loss: 192.894  ; Accuracy: 0.651306
10 Validation loss: 190.365  ; Best loss: 190.365  ; Accuracy: 0.669821
11 Validation loss: 188.214  ; Best loss: 188.214  ; Accuracy: 0.682987
12 Validation loss: 186.358  ; Best loss: 186.358  ; Accuracy: 0.692039
13 Validation los

116 Validation loss: 163.747  ; Best loss: 163.747  ; Accuracy: 0.818761
117 Validation loss: 163.722  ; Best loss: 163.722  ; Accuracy: 0.818556
118 Validation loss: 163.696  ; Best loss: 163.696  ; Accuracy: 0.819173
119 Validation loss: 163.67  ; Best loss: 163.67  ; Accuracy: 0.819379
120 Validation loss: 163.644  ; Best loss: 163.644  ; Accuracy: 0.820202
121 Validation loss: 163.617  ; Best loss: 163.617  ; Accuracy: 0.820202
122 Validation loss: 163.59  ; Best loss: 163.59  ; Accuracy: 0.819996
123 Validation loss: 163.563  ; Best loss: 163.563  ; Accuracy: 0.819996
124 Validation loss: 163.535  ; Best loss: 163.535  ; Accuracy: 0.820613
125 Validation loss: 163.507  ; Best loss: 163.507  ; Accuracy: 0.821436
126 Validation loss: 163.478  ; Best loss: 163.478  ; Accuracy: 0.821642
127 Validation loss: 163.449  ; Best loss: 163.449  ; Accuracy: 0.822259
128 Validation loss: 163.42  ; Best loss: 163.42  ; Accuracy: 0.822464
129 Validation loss: 163.391  ; Best loss: 163.391  ; Acc

232 Validation loss: 161.21  ; Best loss: 161.21  ; Accuracy: 0.834808
233 Validation loss: 161.204  ; Best loss: 161.204  ; Accuracy: 0.834808
234 Validation loss: 161.199  ; Best loss: 161.199  ; Accuracy: 0.835013
235 Validation loss: 161.193  ; Best loss: 161.193  ; Accuracy: 0.835013
236 Validation loss: 161.188  ; Best loss: 161.188  ; Accuracy: 0.834602
237 Validation loss: 161.184  ; Best loss: 161.184  ; Accuracy: 0.834602
238 Validation loss: 161.179  ; Best loss: 161.179  ; Accuracy: 0.834396
239 Validation loss: 161.175  ; Best loss: 161.175  ; Accuracy: 0.83419
240 Validation loss: 161.171  ; Best loss: 161.171  ; Accuracy: 0.834602
241 Validation loss: 161.168  ; Best loss: 161.168  ; Accuracy: 0.834602
242 Validation loss: 161.164  ; Best loss: 161.164  ; Accuracy: 0.834602
243 Validation loss: 161.161  ; Best loss: 161.161  ; Accuracy: 0.834396
244 Validation loss: 161.158  ; Best loss: 161.158  ; Accuracy: 0.834396
245 Validation loss: 161.156  ; Best loss: 161.156  ; 

346 Validation loss: 159.842  ; Best loss: 159.842  ; Accuracy: 0.838922
347 Validation loss: 159.831  ; Best loss: 159.831  ; Accuracy: 0.838716
348 Validation loss: 159.82  ; Best loss: 159.82  ; Accuracy: 0.838716
349 Validation loss: 159.81  ; Best loss: 159.81  ; Accuracy: 0.838511
350 Validation loss: 159.799  ; Best loss: 159.799  ; Accuracy: 0.838099
351 Validation loss: 159.789  ; Best loss: 159.789  ; Accuracy: 0.838099
352 Validation loss: 159.779  ; Best loss: 159.779  ; Accuracy: 0.838099
353 Validation loss: 159.769  ; Best loss: 159.769  ; Accuracy: 0.838099
354 Validation loss: 159.76  ; Best loss: 159.76  ; Accuracy: 0.837893
355 Validation loss: 159.75  ; Best loss: 159.75  ; Accuracy: 0.837688
356 Validation loss: 159.741  ; Best loss: 159.741  ; Accuracy: 0.837688
357 Validation loss: 159.733  ; Best loss: 159.733  ; Accuracy: 0.837482
358 Validation loss: 159.724  ; Best loss: 159.724  ; Accuracy: 0.837482
359 Validation loss: 159.715  ; Best loss: 159.715  ; Accur

460 Validation loss: 159.289  ; Best loss: 159.289  ; Accuracy: 0.838305
461 Validation loss: 159.287  ; Best loss: 159.287  ; Accuracy: 0.838305
462 Validation loss: 159.284  ; Best loss: 159.284  ; Accuracy: 0.838305
463 Validation loss: 159.281  ; Best loss: 159.281  ; Accuracy: 0.838305
464 Validation loss: 159.278  ; Best loss: 159.278  ; Accuracy: 0.838305
465 Validation loss: 159.275  ; Best loss: 159.275  ; Accuracy: 0.838305
466 Validation loss: 159.273  ; Best loss: 159.273  ; Accuracy: 0.838099
467 Validation loss: 159.27  ; Best loss: 159.27  ; Accuracy: 0.838305
468 Validation loss: 159.267  ; Best loss: 159.267  ; Accuracy: 0.838305
469 Validation loss: 159.265  ; Best loss: 159.265  ; Accuracy: 0.838511
470 Validation loss: 159.262  ; Best loss: 159.262  ; Accuracy: 0.838716
471 Validation loss: 159.259  ; Best loss: 159.259  ; Accuracy: 0.838716
472 Validation loss: 159.257  ; Best loss: 159.257  ; Accuracy: 0.838716
473 Validation loss: 159.254  ; Best loss: 159.254  ;

573 Validation loss: 159.085  ; Best loss: 159.081  ; Accuracy: 0.837893
574 Validation loss: 159.087  ; Best loss: 159.081  ; Accuracy: 0.838099
575 Validation loss: 159.089  ; Best loss: 159.081  ; Accuracy: 0.838099
576 Validation loss: 159.092  ; Best loss: 159.081  ; Accuracy: 0.838099
577 Validation loss: 159.095  ; Best loss: 159.081  ; Accuracy: 0.837893
578 Validation loss: 159.099  ; Best loss: 159.081  ; Accuracy: 0.837688
579 Validation loss: 159.104  ; Best loss: 159.081  ; Accuracy: 0.837893
580 Validation loss: 159.111  ; Best loss: 159.081  ; Accuracy: 0.837482
581 Validation loss: 159.119  ; Best loss: 159.081  ; Accuracy: 0.837482
582 Validation loss: 159.129  ; Best loss: 159.081  ; Accuracy: 0.837893
583 Validation loss: 159.142  ; Best loss: 159.081  ; Accuracy: 0.837688
584 Validation loss: 159.157  ; Best loss: 159.081  ; Accuracy: 0.838305
585 Validation loss: 159.175  ; Best loss: 159.081  ; Accuracy: 0.838511
586 Validation loss: 159.193  ; Best loss: 159.081 